# Application programming interfaces: APIs

## Lecture objectives

1. Understand different ways to access data via an API
2. Introduce the `requests` library 
3. Learn how to parse JSON data

One way to get data is to download it manually from a website. You might click through a series of links and then save a `.csv` or similar file to your hard drive.

Another way is through an Application Programming Interface (API). These APIs make it possible to request just the data that you are interested in. Sometimes, this data is static—it will be the same each time you request it. The U.S. Census is a good example. Other APIs are dynamic—for example, bus and train arrival times.

## Example: BART
Many APIs return a format known as JSON. While it seems complicated, it's relatively easy to work with because it's highly structured.

BART provides its [API documentation here](https://api.bart.gov/docs/overview/index.aspx).

How do we get data from the API into Python? The `requests` library is the key to unlocking many interactions with the web.

Let's get the real-time departures from 12th St Oakland City Center station. According to the [documentation](https://api.bart.gov/docs/etd/etd.aspx), we need to pass `orig=12TH`. We can also pass `json=y` to return the results in JSON format.

Most APIs, including that of BART, require a "key" that identifies you to the developer. For serious usage, you'd request your own key. But for experimentation, BART provides a key that anyone can use. 

So how do we construct the request? We simply build a string that contains all of these commands, following the example in the documentation. Then we pass that string to `requests.get()`.

In [1]:
import requests

APIkey = 'MW9S-E7SL-26DU-VV8V'  # the key posted on BART's website
requestString = 'http://api.bart.gov/api/etd.aspx?cmd=etd&orig=12TH&json=y&key='+APIkey

r = requests.get(requestString)

Here, we've made a request to the BART API, and stored the response in the `r` object. 

Let's see what the `r` object includes. Use `r.` and the `tab` autocomplete in Jupyter Notebook to see the different attributes. Or you can call the help function.

In [2]:
help(r)

Help on Response in module requests.models object:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |  
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if

One attribute is `ok`: did the request succeed?

In [3]:
print(r.ok)

True


The `text` attribute shows the text that was returned.

In [4]:
print(r.text)

{"?xml":{"@version":"1.0","@encoding":"utf-8"},"root":{"@id":"1","uri":{"#cdata-section":"http://api.bart.gov/api/etd.aspx?cmd=etd&orig=12TH&json=y"},"date":"04/08/2022","time":"01:42:41 PM PDT","station":[{"name":"12th St. Oakland City Center","abbr":"12TH","etd":[{"destination":"Antioch","abbreviation":"ANTC","limited":"0","estimate":[{"minutes":"13","platform":"3","direction":"North","length":"9","color":"YELLOW","hexcolor":"#ffff33","bikeflag":"1","delay":"0"},{"minutes":"28","platform":"3","direction":"North","length":"10","color":"YELLOW","hexcolor":"#ffff33","bikeflag":"1","delay":"0"},{"minutes":"43","platform":"3","direction":"North","length":"10","color":"YELLOW","hexcolor":"#ffff33","bikeflag":"1","delay":"0"}]},{"destination":"Berryessa","abbreviation":"BERY","limited":"0","estimate":[{"minutes":"3","platform":"2","direction":"South","length":"5","color":"ORANGE","hexcolor":"#ff9933","bikeflag":"1","delay":"61"},{"minutes":"18","platform":"2","direction":"South","length":"5

This looks promising. But how can we get this into a more usable form? The `json` module is the key. It's built-in to Python so you don't have to install anything.

In [5]:
import json

In [6]:
# json.loads() will turn the JSON object into a dictionary
d = json.loads(r.text)
print(type(d))

<class 'dict'>


In [7]:
# remember, a dictionary is a collection of keys and values
# here looks like there are two keys, called ?xml and root
# let's look at the root item
print(d.keys())
print(d['root'])

dict_keys(['?xml', 'root'])
{'@id': '1', 'uri': {'#cdata-section': 'http://api.bart.gov/api/etd.aspx?cmd=etd&orig=12TH&json=y'}, 'date': '04/08/2022', 'time': '01:42:41 PM PDT', 'station': [{'name': '12th St. Oakland City Center', 'abbr': '12TH', 'etd': [{'destination': 'Antioch', 'abbreviation': 'ANTC', 'limited': '0', 'estimate': [{'minutes': '13', 'platform': '3', 'direction': 'North', 'length': '9', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}, {'minutes': '28', 'platform': '3', 'direction': 'North', 'length': '10', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}, {'minutes': '43', 'platform': '3', 'direction': 'North', 'length': '10', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}]}, {'destination': 'Berryessa', 'abbreviation': 'BERY', 'limited': '0', 'estimate': [{'minutes': '3', 'platform': '2', 'direction': 'South', 'length': '5', 'color': 'ORANGE', 'hexcolor': '#ff9933', 'bikeflag': '1', 'delay': '6

In [8]:
# and in turn, 'root' is another dictionary. You could also have seen this because of the curly brackets { }  
print((d['root'].keys()))

dict_keys(['@id', 'uri', 'date', 'time', 'station', 'message'])


The "station" item seems to hold most of the useful information. 

It's a list.

In [9]:
print(type((d['root']['station'])))

<class 'list'>


Of length 1

In [10]:
print(len(d['root']['station']))

1


In [11]:
print(d['root']['station'][0])

{'name': '12th St. Oakland City Center', 'abbr': '12TH', 'etd': [{'destination': 'Antioch', 'abbreviation': 'ANTC', 'limited': '0', 'estimate': [{'minutes': '13', 'platform': '3', 'direction': 'North', 'length': '9', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}, {'minutes': '28', 'platform': '3', 'direction': 'North', 'length': '10', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}, {'minutes': '43', 'platform': '3', 'direction': 'North', 'length': '10', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}]}, {'destination': 'Berryessa', 'abbreviation': 'BERY', 'limited': '0', 'estimate': [{'minutes': '3', 'platform': '2', 'direction': 'South', 'length': '5', 'color': 'ORANGE', 'hexcolor': '#ff9933', 'bikeflag': '1', 'delay': '61'}, {'minutes': '18', 'platform': '2', 'direction': 'South', 'length': '5', 'color': 'ORANGE', 'hexcolor': '#ff9933', 'bikeflag': '1', 'delay': '0'}, {'minutes': '33', 'platform': '2', 'dir

And this is another dictionary! (You can tell by the curly brackets).

Most of the information appears to be in `etd`, which is another list. 

In [12]:
print(d['root']['station'][0]['etd'])

[{'destination': 'Antioch', 'abbreviation': 'ANTC', 'limited': '0', 'estimate': [{'minutes': '13', 'platform': '3', 'direction': 'North', 'length': '9', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}, {'minutes': '28', 'platform': '3', 'direction': 'North', 'length': '10', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}, {'minutes': '43', 'platform': '3', 'direction': 'North', 'length': '10', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}]}, {'destination': 'Berryessa', 'abbreviation': 'BERY', 'limited': '0', 'estimate': [{'minutes': '3', 'platform': '2', 'direction': 'South', 'length': '5', 'color': 'ORANGE', 'hexcolor': '#ff9933', 'bikeflag': '1', 'delay': '61'}, {'minutes': '18', 'platform': '2', 'direction': 'South', 'length': '5', 'color': 'ORANGE', 'hexcolor': '#ff9933', 'bikeflag': '1', 'delay': '0'}, {'minutes': '33', 'platform': '2', 'direction': 'South', 'length': '5', 'color': 'ORANGE', 'hexcolor': 

Each element of the list appears to be a dictionary, giving details of trains to a particular destination. Let's simplify things by pulling this list out to a separate variable.

The `destination` item is self explanatory. The `estimate` item is yet another list!

In [13]:
etd = d['root']['station'][0]['etd']

print(etd[0]['destination'])
print(etd[0]['estimate'])

print(etd[1]['destination'])
print(etd[1]['estimate'])

Antioch
[{'minutes': '13', 'platform': '3', 'direction': 'North', 'length': '9', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}, {'minutes': '28', 'platform': '3', 'direction': 'North', 'length': '10', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}, {'minutes': '43', 'platform': '3', 'direction': 'North', 'length': '10', 'color': 'YELLOW', 'hexcolor': '#ffff33', 'bikeflag': '1', 'delay': '0'}]
Berryessa
[{'minutes': '3', 'platform': '2', 'direction': 'South', 'length': '5', 'color': 'ORANGE', 'hexcolor': '#ff9933', 'bikeflag': '1', 'delay': '61'}, {'minutes': '18', 'platform': '2', 'direction': 'South', 'length': '5', 'color': 'ORANGE', 'hexcolor': '#ff9933', 'bikeflag': '1', 'delay': '0'}, {'minutes': '33', 'platform': '2', 'direction': 'South', 'length': '5', 'color': 'ORANGE', 'hexcolor': '#ff9933', 'bikeflag': '1', 'delay': '0'}]


We can print this more nicely using the `.format()` method for a string. The curly braces `{}` are placeholders for the items to be inserted.

In [14]:
print('Train to {} is arriving in {} minutes'.format(etd[0]['destination'], etd[0]['estimate'][0]['minutes']))

Train to Antioch is arriving in 13 minutes


To make this easier to work with, we can convert to a pandas `DataFrame`. This doesn't always work, but is worth a try.

In [15]:
import pandas as pd
df = pd.DataFrame(etd[0]['estimate'])
df

,minutes,platform,direction,length,color,hexcolor,bikeflag,delay
0,13,3,North,9,YELLOW,#ffff33,1,0
1,28,3,North,10,YELLOW,#ffff33,1,0
2,43,3,North,10,YELLOW,#ffff33,1,0


<div class="alert alert-block alert-info">
<strong>Exercise:</strong> How would you calculate the mean headway (time between trains)?
</div>

Now, let's generalize. The API above takes a string of a particular form. So we can write a function that gets the times for any station. All we are doing here is taking the code above, putting it into a function, and making the station id an argument to that function.

Note the use of `.format()` to put the station and API key into the string.

Also note that it only gives the times for the first destination. 

In [16]:
def getArrivalTimes(station):
    """Returns a dataframe of arrival times for a given stop id"""
    requestString = 'http://api.bart.gov/api/etd.aspx?cmd=etd&orig={}&json=y&key={}'.format(station, APIkey)
    r = requests.get(requestString)
    d = json.loads(r.text)
    etd = d['root']['station'][0]['etd']
    print('Trains from {} to {}'.format(station, etd[0]['destination']))
    df = pd.DataFrame(etd[0]['estimate'])
    return df


getArrivalTimes('12TH')

Trains from 12TH to Antioch


,minutes,platform,direction,length,color,hexcolor,bikeflag,delay
0,11,3,North,9,YELLOW,#ffff33,1,0
1,26,3,North,10,YELLOW,#ffff33,1,0
2,41,3,North,10,YELLOW,#ffff33,1,0


We can get a list of stations using the `stns` command, according to the [documentation](https://api.bart.gov/docs/stn/stns.aspx).

Then let's try another station using our handy function.

In [17]:
requestString = 'http://api.bart.gov/api/stn.aspx?cmd=stns&key={}'.format(APIkey)
r = requests.get(requestString)
print(r.text)

<?xml version="1.0" encoding="utf-8"?><root><uri><![CDATA[http://api.bart.gov/api/stn.aspx?cmd=stns]]></uri><stations><station><name>12th St. Oakland City Center</name><abbr>12TH</abbr><gtfs_latitude>37.803768</gtfs_latitude><gtfs_longitude>-122.271450</gtfs_longitude><address>1245 Broadway</address><city>Oakland</city><county>alameda</county><state>CA</state><zipcode>94612</zipcode></station><station><name>16th St. Mission</name><abbr>16TH</abbr><gtfs_latitude>37.765062</gtfs_latitude><gtfs_longitude>-122.419694</gtfs_longitude><address>2000 Mission Street</address><city>San Francisco</city><county>sanfrancisco</county><state>CA</state><zipcode>94110</zipcode></station><station><name>19th St. Oakland</name><abbr>19TH</abbr><gtfs_latitude>37.808350</gtfs_latitude><gtfs_longitude>-122.268602</gtfs_longitude><address>1900 Broadway</address><city>Oakland</city><county>alameda</county><state>CA</state><zipcode>94612</zipcode></station><station><name>24th St. Mission</name><abbr>24TH</abbr>

In [18]:
getArrivalTimes('CIVC')

Trains from CIVC to Antioch


,minutes,platform,direction,length,color,hexcolor,bikeflag,delay
0,10,2,North,10,YELLOW,#ffff33,1,0
1,25,2,North,10,YELLOW,#ffff33,1,0
2,40,2,North,10,YELLOW,#ffff33,1,0


<div class="alert alert-block alert-info">
<strong>Let's recap.</strong> What did we just do?
<ul>  
<li>We constructed a text string following the API documentation, and passed that string to `requests`</li>
<li>We did some step-by-step detective work to convert the output into a usable format</li>
<li>We combined all of these steps into a function</li>
</uli>
</div>

Let's focus on the first step, and try more commands from the BART API, [such as returning fare information](https://api.bart.gov/docs/sched/fare.aspx).

We see that the string begins like this:

`http://api.bart.gov/api/sched.aspx?cmd=fare`

Then we add the various inputs separated by `?`.

For example, the fare from `12TH` to `CIVC` is as follows.

In [19]:
requestString = 'http://api.bart.gov/api/sched.aspx?cmd=fare&orig=12TH&dest=CIVC&key={}&json=y'.format(APIkey)
r = requests.get(requestString)
print(r.text)

{"?xml":{"@version":"1.0","@encoding":"utf-8"},"root":{"uri":{"#cdata-section":"http://api.bart.gov/api/sched.aspx?cmd=fare&orig=12TH&dest=CIVC&json=y"},"origin":"12TH","destination":"CIVC","trip":{"fare":"3.70","discount":{"clipper":"1.35"}},"fares":{"@level":"normal","fare":[{"@amount":"3.70","@class":"clipper","@name":"Clipper"},{"@amount":"2.95","@class":"start","@name":"Clipper START"},{"@amount":"1.35","@class":"rtcclipper","@name":"Senior/Disabled Clipper"},{"@amount":"1.85","@class":"student","@name":"Youth Clipper"}]},"message":""}}


Note that the output is the same as if you paste the string into a web browser.

In [20]:
print(requestString)

http://api.bart.gov/api/sched.aspx?cmd=fare&orig=12TH&dest=CIVC&key=MW9S-E7SL-26DU-VV8V&json=y


An alternative and more elegant way of calling requests is to put all the inputs (parameters) into a dictionary. This version is identical to the previous API call.

In [21]:
requestString = 'http://api.bart.gov/api/sched.aspx'.format(APIkey)
params = {'cmd':'fare',
          'orig':'12TH',
          'dest':'CIVC',
          'key':APIkey,
          'json':'y'}
r = requests.get(requestString, params=params)
print(r.text)

{"?xml":{"@version":"1.0","@encoding":"utf-8"},"root":{"uri":{"#cdata-section":"http://api.bart.gov/api/sched.aspx?cmd=fare&orig=12TH&dest=CIVC&json=y"},"origin":"12TH","destination":"CIVC","trip":{"fare":"3.70","discount":{"clipper":"1.35"}},"fares":{"@level":"normal","fare":[{"@amount":"3.70","@class":"clipper","@name":"Clipper"},{"@amount":"2.95","@class":"start","@name":"Clipper START"},{"@amount":"1.35","@class":"rtcclipper","@name":"Senior/Disabled Clipper"},{"@amount":"1.85","@class":"student","@name":"Youth Clipper"}]},"message":""}}


<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Explore some of BART's other API collections at the same link above
</div>

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Many APIs are just URLs. You can compose the URL as a string.</li>
  <li>JSON is the typical format of the returned data, but you will often need to experiment.</li>
  <li>Be nice! Some APIs will ask you to register. Most will kick you off if you make too many requests.</li>
</ul>
</div>